In [1]:
# ORM -> SqlAlchemy
# Database Adapter -> Psycopg2

In [2]:
!pip install psycopg2


In [4]:
import psycopg2

connection = psycopg2.connect(
    dbname='testdb',
    user='postgres',
    password='1234',
    host='193.203.191.79',
    port='32001'
)

query = "Select * from users"

In [5]:
import pandas as pd

df = pd.read_sql(query, connection)
df.head(3)

C:\Users\PC1\AppData\Local\Temp\ipykernel_37320\1197474191.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


,id,name,email,review,rating
0,1,Raymond Howard,erodriguez@example.net,Partner trouble serious sign bar follow. Assum...,6.8
1,2,Robert Mccarthy,williamsjames@example.org,Truth region personal how position technology....,2.1
2,3,Sean Russell,linda46@example.org,Figure news significant stock. Scientist voice...,2.9


In [6]:
connection.close()

In [7]:
df.shape

(4011, 5)

In [8]:
texts = df['review'].values
ratings = df['rating'].values

In [14]:
#!pip install scikit-learn

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(texts,ratings,test_size=0.3, random_state=42)

In [16]:
#!pip install tensorflow

In [18]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=10000, oov_token='<OOV>') # Out-of vocabulary
tokenizer.fit_on_texts(X_train)

In [25]:
#tokenizer.word_index # Answer simple figure  => [182, 184, 173]

In [26]:
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

In [27]:
#X_train_seq[0], X_train[0]

In [28]:
# Padding -> (50) => 50'den kısa cümleler için başına ya da sonuna 0 ekle. (0 => Boşluk) || 50'den büyükse 50şerlik arrayleri böl.
from tensorflow.keras.preprocessing.sequence import pad_sequences

X_train_pad = pad_sequences(X_train_seq, maxlen=50, padding="post")
X_test_pad = pad_sequences(X_test_seq, maxlen=50, padding="post")
#

In [31]:
#Ödev 1 => Aktivasyon fonksiyonu nedir, ne işe yarar? Aktivasyon fonksiyon türleri nelerdir?
#Ödev 2 => Backpropagation nedir? Optimizer nedir? Optimizer türleri?
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=10000, output_dim=64, input_length=50), #Kelimeleri vektörlere çevirir.
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)), # Bidirectional -> Soldan sağa ve sağdan sola, #LSTM => Long short-term Memory -> 128 nöronluk çıktı.
    tf.keras.layers.Dense(32, activation="relu"), # Non-linear bi yapı kurarak gerçek hayata yaklaşmak. max(0,z)
    tf.keras.layers.Dropout(0.4), # Her çalıştırmada rastgele verilen oranda nöronu kapatır.
    tf.keras.layers.Dense(1, activation="linear") # linear => f(x)=x # 1-> Çıktı adet sayısı.
])
# Model eğitimi öncesi özellikleri belirleme.
model.compile(loss="mean_squared_error", metrics=['mae'], optimizer="adam")

history = model.fit(X_train_pad, y_train, epochs=3, validation_data=(X_test_pad, y_test), batch_size=2)


Epoch 1/3
1404/1404 ━━━━━━━━━━━━━━━━━━━━ 12s 8ms/step - loss: 11.0517 - mae: 2.7596 - val_loss: 6.7463 - val_mae: 2.2362
Epoch 2/3
1404/1404 ━━━━━━━━━━━━━━━━━━━━ 11s 8ms/step - loss: 7.9405 - mae: 2.3548 - val_loss: 7.0909 - val_mae: 2.2703
Epoch 3/3
1404/1404 ━━━━━━━━━━━━━━━━━━━━ 11s 8ms/step - loss: 6.5639 - mae: 2.0682 - val_loss: 8.2217 - val_mae: 2.3998


In [32]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ (None, 50, 64)              │         640,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_1 (Bidirectional)      │ (None, 128)                 │          66,048 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 32)                  │           4,128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,130,629 (8.13 MB)

 Trainable params: 710,209 (2.71 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,420,420 (5.42 MB)